In [1]:
import ipyparallel as ipp
c = ipp.Client(profile='mpi')
print(c.ids)
view = c[:]
view.activate()

[0, 1, 2, 3, 4, 5]


In [2]:
%%px --block 
from mpi4py import MPI
try:
    import numpy
except ImportError:
    raise SystemExit

# --------------------------------------------------------------------

n = 5 * MPI.COMM_WORLD.Get_size()

# compute number of processes and myrank
p = MPI.COMM_WORLD.Get_size()
myrank = MPI.COMM_WORLD.Get_rank()

# compute size of local block
m = n//p
if myrank < (n - p * m):
    m = m + 1

#compute neighbors
if myrank == 0:
    left = MPI.PROC_NULL
else:
    left = myrank - 1
if myrank == p - 1:
    right = MPI.PROC_NULL
else:
    right = myrank + 1

# allocate local arrays
A = numpy.empty((n+2, m+2), dtype='d', order='F')
B = numpy.empty((n, m),     dtype='d', order='F')

A.fill(1)
A[0, :] = A[-1, :] = 0
A[:, 0] = A[:, -1] = 0

# main loop
converged = False
#while not converged:
for i in range(10):
    # compute,  B = 0.25 * ( N + S + E + W)
    N, S = A[:-2, 1:-1], A[2:, 1:-1]
    E, W = A[1:-1, :-2], A[1:-1, 2:]
    numpy.add(N, S, B)
    numpy.add(E, B, B)
    numpy.add(W, B, B)
    B *= 0.25
    A[1:-1, 1:-1] = B
    # communicate
    tag = 0
    MPI.COMM_WORLD.Sendrecv([B[:, -1], MPI.DOUBLE], right, tag,
                            [A[:,  0], MPI.DOUBLE], left,  tag)
    MPI.COMM_WORLD.Sendrecv((B[:,  0], MPI.DOUBLE), left,  tag,
                            (A[:, -1], MPI.DOUBLE), right, tag)
    # convergence
    myconv = numpy.allclose(B, 0)
    loc_conv = numpy.asarray(myconv, dtype='i')
    glb_conv = numpy.asarray(0, dtype='i')
    MPI.COMM_WORLD.Allreduce([loc_conv, MPI.INT],
                             [glb_conv, MPI.INT],
                             op=MPI.LAND)
    converged = bool(glb_conv)
print("OK")

[stdout:0] OK
[stdout:1] OK
[stdout:2] OK
[stdout:3] OK
[stdout:4] OK
[stdout:5] OK


In [6]:
view['A'][3]

array([[ 0.4308033 ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.4308033 ],
       [ 0.66515636,  0.4308033 ,  0.34909916,  0.33295631,  0.34909916,
         0.4308033 ,  0.66515636],
       [ 0.79619503,  0.66515636,  0.60678768,  0.58583832,  0.60678768,
         0.66515636,  0.79619503],
       [ 0.8567028 ,  0.79619503,  0.76002121,  0.75399208,  0.76002121,
         0.79619503,  0.8567028 ],
       [ 0.88446808,  0.8567028 ,  0.84567165,  0.84032059,  0.84567165,
         0.8567028 ,  0.88446808],
       [ 0.89346218,  0.88446808,  0.88132858,  0.88274765,  0.88132858,
         0.88446808,  0.89346218],
       [ 0.89640236,  0.89346218,  0.89592743,  0.8973608 ,  0.89592743,
         0.89346218,  0.89640236],
       [ 0.89698792,  0.89640236,  0.89987755,  0.9023056 ,  0.89987755,
         0.89640236,  0.89698792],
       [ 0.89710331,  0.89698792,  0.90092373,  0.90334511,  0.90092373,
         0.89698792,  0.89710331],
       [ 0.8971138 ,  0.8971

In [5]:
view['glb_conv'][3]

array(0, dtype=int32)